# Tabular Playground - Jan 2021 - AutoML

# 1. Librerias a utilizar

In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
input_path = Path('/kaggle/input/tabular-playground-series-jan-2021/')


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


#Pipeline
from sklearn.pipeline import Pipeline

#Tuning de Parametros
from sklearn.model_selection import GridSearchCV

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoost, Pool

import h2o
print(h2o.__version__)
from h2o.automl import H2OAutoML

h2o.init(max_mem_size='16G')

/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-jan-2021/train.csv
/kaggle/input/tabular-playground-series-jan-2021/test.csv


3.32.0.2
Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.9.1" 2020-11-04; OpenJDK Runtime Environment (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp17iaj0qx
  JVM stdout: /tmp/tmp17iaj0qx/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp17iaj0qx/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.2
H2O_cluster_version_age:,1 month and 26 days
H2O_cluster_name:,H2O_from_python_unknownUser_kmz300
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,16 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


# 2. Dataset de entrenamiento / Dataset de testing

In [2]:
#Dataset con los datos de entrenamiento

train = h2o.import_file('../input/tabular-playground-series-jan-2021/train.csv')
train.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%


id,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,target
1,0.67039,0.8113,0.643968,0.291791,0.284117,0.855953,0.8907,0.285542,0.558245,0.779418,0.921832,0.866772,0.878733,0.305411,7.24304
3,0.388053,0.621104,0.686102,0.501149,0.64379,0.449805,0.510824,0.580748,0.418335,0.432632,0.439872,0.434971,0.369957,0.369484,8.20333
4,0.83495,0.227436,0.301584,0.293408,0.606839,0.829175,0.506143,0.558771,0.587603,0.823312,0.567007,0.677708,0.882938,0.303047,7.77609
5,0.820708,0.160155,0.546887,0.726104,0.282444,0.785108,0.752758,0.823267,0.574466,0.580843,0.769594,0.818143,0.914281,0.279528,6.95772
8,0.935278,0.421235,0.303801,0.880214,0.66561,0.830131,0.487113,0.604157,0.874658,0.863427,0.983575,0.900464,0.935918,0.435772,7.95105
9,0.352623,0.258867,0.327373,0.802627,0.284219,0.296886,0.209743,0.27371,0.308018,0.235851,0.27876,0.251406,0.339135,0.293129,7.34687
11,0.259096,0.803934,0.5809,0.322884,0.984705,0.378247,0.432821,0.562059,0.290965,0.316543,0.219192,0.326977,0.458653,0.2443,8.61624
12,0.48161,0.805446,0.235836,0.365011,0.734725,0.404825,0.465061,0.578794,0.418807,0.451958,0.45872,0.586586,0.321026,0.834694,6.8339
13,0.66792,0.776422,0.255154,0.869598,0.678067,0.475274,0.49329,0.638779,0.491023,0.617451,0.413352,0.485558,0.474055,0.608374,8.54079
15,0.937751,0.236482,0.216445,0.476463,0.283642,0.456763,0.316416,0.785602,0.695845,0.370941,0.436835,0.826278,0.856748,0.418593,7.70105


In [3]:
train.shape

(300000, 16)

In [4]:
#Dataset con los datos de testing

test = h2o.import_file('../input/tabular-playground-series-jan-2021/test.csv')
test.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%


id,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,0.3536,0.73878,0.600939,0.293377,0.285691,0.458006,0.620704,0.422249,0.369203,0.435727,0.55054,0.699134,0.286864,0.364515
2,0.907222,0.189756,0.215531,0.869915,0.301333,0.528958,0.390351,0.521112,0.794779,0.79858,0.446475,0.449037,0.916964,0.513002
6,0.179287,0.355353,0.623972,0.437812,0.282476,0.320826,0.386789,0.776422,0.222268,0.229102,0.211913,0.222651,0.327164,0.827941
7,0.359385,0.181049,0.551368,0.206386,0.280763,0.482076,0.506677,0.362793,0.379737,0.345686,0.445276,0.518485,0.299028,0.598166
10,0.335791,0.682607,0.676481,0.219465,0.282861,0.581721,0.748639,0.350158,0.448915,0.506878,0.817721,0.805895,0.790591,0.249275
14,0.256414,0.621835,0.959441,0.913052,0.387511,0.31462,0.322014,0.614673,0.292548,0.899578,0.278104,0.274086,0.418178,0.715106
16,0.485888,0.359113,0.613006,0.257908,0.283429,0.646103,0.963301,0.311508,0.58126,0.638479,0.840254,0.854393,0.893814,0.269365
17,0.673931,0.734818,0.361756,0.567235,0.284732,0.453174,0.515395,0.789382,0.714804,0.265037,0.323776,0.507847,0.539327,0.849788
18,0.926054,0.619843,0.356893,0.868551,0.284436,0.865728,0.456673,0.729383,0.73935,0.867766,0.906833,0.963105,0.901933,0.826835
19,0.497196,0.619562,0.623283,0.323464,0.5031,0.457458,0.447466,0.612733,0.40986,0.438677,0.330754,0.348405,0.355242,0.259408


In [5]:
test.shape

(200000, 15)

In [6]:
#Dataset para realizar el submit

submission = h2o.import_file('../input/tabular-playground-series-jan-2021/sample_submission.csv')
submission.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%


id,target
0,0.5
2,0.5
6,0.5
7,0.5
10,0.5
14,0.5
16,0.5
17,0.5
18,0.5
19,0.5


In [7]:
submission.shape

(200000, 2)

# 3. AutoML H2O

In [8]:
folds = 10

model = H2OAutoML(max_runtime_secs=10800, stopping_metric = "RMSE",sort_metric="RMSE",seed = 1, project_name = "autoML",nfolds = folds)

In [9]:
x = train[1:-1].columns
y = "target"

In [10]:
model.train(x=x,y=y,training_frame = train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [11]:
resumen_leaderboard = model.leaderboard
resumen_leaderboard

model_id,rmse,mean_residual_deviance,mse,mae,rmsle
GBM_5_AutoML_20210112_190700,0.701648,0.492309,0.492309,0.588243,0.0797924
GBM_grid__1_AutoML_20210112_190700_model_3,0.70169,0.492368,0.492368,0.588545,0.079795
GBM_grid__1_AutoML_20210112_190700_model_2,0.702089,0.492929,0.492929,0.589004,0.0798399
GBM_4_AutoML_20210112_190700,0.702134,0.492993,0.492993,0.588601,0.0798479
GBM_3_AutoML_20210112_190700,0.702228,0.493124,0.493124,0.589473,0.079862
XGBoost_grid__1_AutoML_20210112_190700_model_5,0.702888,0.494052,0.494052,0.589029,0.0799212
GBM_2_AutoML_20210112_190700,0.70301,0.494223,0.494223,0.59031,0.0799489
XGBoost_3_AutoML_20210112_190700,0.703145,0.494413,0.494413,0.589159,0.07995
GBM_1_AutoML_20210112_190700,0.704162,0.495845,0.495845,0.591477,0.0800746
GBM_grid__1_AutoML_20210112_190700_model_5,0.704239,0.495952,0.495952,0.588127,0.0800728


In [12]:
model.leader

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_5_AutoML_20210112_190700


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,33.0,33.0,483877.0,15.0,15.0,15.0,787.0,1472.0,1163.2122




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 0.4400576021788136
RMSE: 0.6633683759260866
MAE: 0.5562438161977132
RMSLE: 0.07550849319383844
Mean Residual Deviance: 0.4400576021788136

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 0.4923093648231128
RMSE: 0.701647607295224
MAE: 0.5882432293145161
RMSLE: 0.0797923710952454
Mean Residual Deviance: 0.4923093648231128

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid,cv_6_valid,cv_7_valid,cv_8_valid,cv_9_valid,cv_10_valid
0,mae,0.58824325,0.0023599127,0.5878458,0.5873954,0.5859618,0.5861897,0.59283334,0.5893796,0.58579457,0.5866639,0.58933556,0.59103256
1,mean_residual_deviance,0.49230936,0.003719423,0.49168873,0.49121934,0.48991066,0.48853868,0.49955824,0.49451852,0.48767397,0.48967624,0.49447495,0.49583432
2,mse,0.49230936,0.003719423,0.49168873,0.49121934,0.48991066,0.48853868,0.49955824,0.49451852,0.48767397,0.48967624,0.49447495,0.49583432
3,r2,0.08384802,0.0023131273,0.08782717,0.085767366,0.08235623,0.080462456,0.085367575,0.08198664,0.08167602,0.083921805,0.085870214,0.08324473
4,residual_deviance,0.49230936,0.003719423,0.49168873,0.49121934,0.48991066,0.48853868,0.49955824,0.49451852,0.48767397,0.48967624,0.49447495,0.49583432
5,rmse,0.7016431,0.0026477391,0.7012052,0.7008704,0.69993615,0.6989554,0.7067943,0.7032201,0.69833654,0.6997687,0.70318913,0.704155
6,rmsle,0.07979068,5.4688257E-4,0.07961502,0.079641566,0.079620406,0.07932524,0.081224516,0.079858765,0.079401046,0.07943181,0.07977087,0.0800176



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2021-01-12 19:47:07,4 min 28.963 sec,0.0,0.733070,0.619374,0.537391
1,,2021-01-12 19:47:11,4 min 32.493 sec,5.0,0.711341,0.601222,0.506006
2,,2021-01-12 19:47:14,4 min 35.752 sec,10.0,0.697119,0.588317,0.485976
3,,2021-01-12 19:47:17,4 min 38.546 sec,15.0,0.686836,0.578553,0.471744
4,,2021-01-12 19:47:20,4 min 41.338 sec,20.0,0.678956,0.570948,0.460981
5,,2021-01-12 19:47:23,4 min 44.128 sec,25.0,0.672088,0.564465,0.451703
6,,2021-01-12 19:47:26,4 min 47.238 sec,30.0,0.666296,0.558968,0.443950
7,,2021-01-12 19:47:27,4 min 48.858 sec,33.0,0.663368,0.556244,0.440058



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,cont4,14447.833984,1.000000,0.102055
1,cont13,12527.454102,0.867082,0.088490
2,cont3,12349.002930,0.854730,0.087230
3,cont2,11408.712891,0.789649,0.080588
4,cont12,10547.873047,0.730066,0.074507
5,cont7,10228.794922,0.707981,0.072253
6,cont14,10013.279297,0.693064,0.070731
7,cont10,9964.435547,0.689684,0.070386
8,cont1,9376.439453,0.648986,0.066232
9,cont6,9213.111328,0.637681,0.065079


In [13]:
pred = model.leader.predict(test)
pred.head()

gbm prediction progress: |████████████████████████████████████████████████| 100%


predict
7.84362
7.88197
7.86651
8.22786
8.09289
8.00631
8.03086
7.72062
7.82639
7.76747


# 5. Generacion del Submit

In [14]:
submission = pd.read_csv('../input/tabular-playground-series-jan-2021/sample_submission.csv')
submission.shape

(200000, 2)

In [15]:
submission['target'] = pred.as_data_frame().values

In [16]:
submission.to_csv('autoML.csv',index=False)